# XAS Workflow Task 2

This notebook contains the second task of the XAS processing workflow. 

The break up of the task consist of the following steps 

|Task                            | Input                                         | Output
|-------------                   |-------------                                  |-----  
| Curve fitting||
| 1. Import data                |File: FeS2_larch.prj                              |
| 2. Import Crystal data        |File: FeS2.inp                                 |
| 3. Calculate Paths(Atoms+FEFF)||
| 4. Set path parameters        | Parameters:                                   |
|                                 |    amp  = 1                                   |
|                                 |    enot = 0                                   |
|                                 |    delr = 0                                   |
|                                 |    ss   = 0.003                               |
| 5. Select paths                 |                                               |
| 6. Run Fit                    |                                               |
| 7. Save project               ||
| 8. Verify fit results         ||
| 8.1 If not OK revise parameners and refit (go to 2.4)||
| 8.2 If OK Save project and outputs|                                           |File: FeS2_01.fpj

For more details about larch, see https://xraypy.github.io/xraylarch/xafs/


## Libraries
Libraries required for running fit on the XAS data.


In [1]:
# managing athena files
from larch.io import create_athena, read_athena, extract_athenagroup
# calculate pre-edge and post edge for normalisation
from larch.xafs import pre_edge
# perform background removal
from larch.xafs import autobk
# calculate fourier transform
from larch.xafs import xftf

# File handling
from pathlib import Path

#plotting library
%matplotlib inline
import matplotlib.pyplot as plt

#library for writing to log
import logging

## Custom Functions

The following cell contains the defined functions (methods) for processing XAS files.

- **set_logger** intialises the logging.
- **get_files_list** returns a list of files in the directory matching the given file pattern.
- **calc_with_defaults ** recalculates mu, normal, pre-edge for each group being processed.
    


In [2]:
 #######################################################
# |                Initialise log file                | #
# V  provide the path and name of the log file to use V #
 #######################################################
    
def set_logger(log_file):
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=log_file, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)

 #######################################################
# |                  Get a list of files              | #
# V       provide the path and pattern to match       V #
 #######################################################
    
#reading all files with the same extension files from a dir
def get_files_list(source_dir, f_pattern):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob(f_pattern)):
        i_counter += 1
        files_list.append(filepath)
    return files_list

 #######################################################
# |         Athena recalculates everything so we      | #
# |      need to create a function that calculates    | #
# V               all for each new group              V #
 #######################################################

def calc_with_defaults(xafs_group):
    # calculate mu and normalise with background extraction
    # should let the user specify the colums for i0, it, mu, iR. 
    if not hasattr(xafs_group, 'mu'):
        xafs_group = get_mu(xafs_group)
    # calculate pre-edge and post edge and add them to group
    # need to read parameters for pre-edge before background calculation with  
    # defaul values undo the work of previous step (setting pre-edge limits).
    pre_edge(xafs_group, pre1=xafs_group.bkg_params.pre1, pre2=xafs_group.bkg_params.pre2)
    #pre_edge(xafs_group)
    # perform background removal
    autobk(xafs_group) # using defaults so no additional parameters are passed
    # calculate fourier transform
    xftf(xafs_group)#, kweight=0.5, kmin=3.0, kmax=12.871, dk=1, kwindow='Hanning')
    return xafs_group


 #######################################################
# |       The code for plotting Nmu vs E repeats      | #
# |   so it is useful to have a plotting function     | #
# V            to reduce duplicated code              V #
 #######################################################
# plot flat normalised mu vs Energy for selected groups
def plot_Nxmu_E(athena_project, group_keys, group_names,
                            title = "Normalised $\mu$ vs E", xlimits = None,
                            ylimits = None):    
    # plot mu vs flat normalised mu for selected groups
    for group_key in group_keys:
        gr_0 = extract_athenagroup(athena_project._athena_groups[group_key])
        # recalculate normalisation
        calc_with_defaults(gr_0)
        plt.plot(gr_0.energy, gr_0.flat, label=group_names[group_key])

    # set plot format
    plt.xlabel("Energy")
    plt.ylabel("Normalised $\mu$" )
    plt.title(title)
    plt.grid(linestyle=':', linewidth=1) #show and format grid
    if xlimits != None:
        plt.xlim(xlimits[0],xlimits[1])
    if ylimits != None:
        plt.ylim(ylimits[0],ylimits[1])
    plt.legend()
    return plt, gr_0


# show plot of normalised data
def plot_normalised(xafs_group):
        plt.plot(xafs_group.energy, xafs_group.mu, label=xafs_group.filename) # plot mu in blue
        plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
        plt.xlabel('Energy (eV)') # label y graph
        plt.ylabel('x$\mu$(E)') # label y axis
        plt.title("pre-edge and post_edge fitting to $\mu$")
        plt.legend() # show legend
        return plt

## Input parameters (variables)

The variables in the next cell indicate where to get the data from, the pattern of the files to process and the number of files to process. These can be changed to process different datasets.


In [14]:
# variables that can be changed to process different datasets
data_path = ".\\rh4co\\"
file_pattern = "*.prj"
f_prefix = "rh4co"
top_count = 4
i_count = 0
show_graph = True # False to prevent showing graphs

## Main function
The code in the cell below performs the bulk of the processing for task 2 calling functions defined above and using the input parameters.

In [ ]:
# create the path for storing results
base_path = Path("./" , f_prefix+"_fit")
Path(base_path).mkdir(parents=True, exist_ok=True)

log_file = Path("./",base_path,"process.log")
print(log_file)
# set path for log
set_logger(log_file)

# get the list of files to process
source_path = Path(data_path)
files_list = get_files_list(source_path, file_pattern)
xas_data = {}

logging.info("Started processing")
for a_file in files_list:
    project_name = a_file.name
    data_prj = read_athena(a_file)
    group_keys = list(data_prj._athena_groups.keys())

    #group_names = {}

    #for group_key in group_keys:
    #    group_names[group_key] = group_key

    data_group = extract_athenagroup(data_prj._athena_groups[group_keys[0]])
    if show_graph:    
        # plot normalised mu on energy
        # plot mu vs flat normalised mu for selected groups
        #plt, data_group = plot_Nxmu_E(data_prj, group_keys, group_names)
        plt = plot_normalised(data_group)
        plt.show()
        
        
    i_count +=1
    
    logging.info("Processed file: "+ str(i_count) +" " + group_keys[0])
    
    if i_count == top_count:
        break
       
logging.info("Finished processing")

[root] INFO : Started processing


rh4co_fit\process.log


[matplotlib.font_manager] DEBUG : findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=12.0.
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'STIXGeneral' (STIXGeneralBolIta.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans Display' (DejaVuSansDisplay.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'cmsy10' (cmsy10.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'STIXNonUnicode' (STIXNonUni.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Serif' (DejaVuSerif-BoldItalic.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'STIXSizeOneSym' (STIXSizOneSymBol.ttf)

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Cambria' (cambriaz.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Gentium Basic' (GenBasBI.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Leelawadee UI' (LeelUIsl.ttf) normal normal 350 normal>) = 10.0975
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Calisto MT' (CALISTI.TTF) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono-Oblique.ttf) oblique normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Arial' (arialbi.ttf) italic normal 700 normal>) = 7.698636363636363
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Consolas' (consolai.ttf) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Bodoni MT' (BOD_CR.TTF) normal normal 400 condensed>) = 10.25
[matplotlib.font_

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Miriam Mono CLM' (MiriamMonoCLM-Book.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Caladea' (Caladea-Bold.ttf) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Carlito' (Carlito-Italic.ttf) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Broadway' (BROADW.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Old English Text MT' (OLDENGL.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Palatino Linotype' (pala.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Bookshelf Symbol 7' (BSSYM7.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Segoe UI' (segoeuiz.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font

...

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Serif' (DejaVuSerif-BoldItalic.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'STIXSizeOneSym' (STIXSizOneSymBol.ttf) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'cmtt10' (cmtt10.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'STIXGeneral' (STIXGeneralItalic.ttf) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'STIXNonUnicode' (STIXNonUniBol.ttf) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono-BoldOblique.ttf) oblique normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSan

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Arial' (arialbi.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Consolas' (consolai.ttf) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Bodoni MT' (BOD_CR.TTF) normal normal 400 condensed>) = 10.25
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans' (DejaVuSansCondensed-Bold.ttf) normal normal 700 condensed>) = 10.535
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Noto Sans Arabic UI' (NotoSansArabicUI-Regular.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Perpetua' (PERB____.TTF) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Noto Sans' (NotoSans-Bold.ttf) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Noto Serif' (NotoSerif-LightItalic.ttf) italic normal 300 normal>) 

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Palatino Linotype' (pala.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Bookshelf Symbol 7' (BSSYM7.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Segoe UI' (segoeuiz.ttf) italic normal 700 normal>) = 11.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Calibri' (calibrii.ttf) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Frank Ruhl Hofshi' (FrankRuhlHofshi-Regular.otf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'High Tower Text' (HTOWERT.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'DejaVu Sans' (DejaVuSansCondensed-Oblique.ttf) oblique normal 400 condensed>) = 11.25
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Nirmala UI' (NirmalaB.ttf) normal normal 700 normal>) =

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Sitka Small' (Sitka.ttc) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Noto Naskh Arabic' (NotoNaskhArabic-Regular.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Verdana' (verdana.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Viner Hand ITC' (VINERITC.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Source Code Pro' (SourceCodePro-LightIt.ttf) italic normal 300 normal>) = 11.145
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Poor Richard' (POORICH.TTF) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Brush Script MT' (BRUSHSCI.TTF) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Rockwell Condensed' (ROCCB___.TTF) normal normal 700 condensed>) 

[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Noto Sans Hebrew' (NotoSansHebrew-Bold.ttf) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Ink Free' (Inkfree.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Linux Libertine G' (LinLibertine_RZ_G.ttf) normal normal 600 normal>) = 10.24
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Cambria' (cambriab.ttf) normal normal 700 normal>) = 10.335
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Detente' (Detente_.ttf) normal normal 5 normal>) = 10.42525
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Source Sans Pro' (SourceSansPro-It.ttf) italic normal 400 normal>) = 11.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Impact' (impact.ttf) normal normal 400 normal>) = 10.05
[matplotlib.font_manager] DEBUG : findfont: score(<Font 'Bell MT' (BELLB.TTF) normal normal 700 normal>) = 10.335
[matplotlib.font_ma

In [ ]:
list(data_prj._athena_groups.keys())
group_names
#data_prj._athena_groups[group_keys[0]]

## Atoms and FEFF 

Larch does larch does not include a means for running atoms. Atoms is needed to get input for feff and calculate paths. Currently, the fastest option is to run Artemis to obtain the input (.inp) file for feff from a crystal file ('.cif' or '.inp')

The code below shows how subprocess can be used to call perl, execute a small perl script that runs Artemis Atoms, and saves the output file ('inp') in a new directory.

The file can then be used to run FEFF from Larch to calculate scattering paths.

In [ ]:
# larch does not include a means for running atoms
# need to get input for feff and calculate paths

# currently the fastest option is to run Artemis to
# obtain the input (.inp) file for feff from a '.cif'
# or '.inp' file

# get subprocess to run perl script
import subprocess

# get the input file
var = "FeS2.inp"

retcode = subprocess.call(["perl", "feff_inp.pl", var])
if retcode == 0:
    print("Passed!")
else:
    print("Failed!")

# run feff and get the paths
from larch.xafs.feffrunner import feff6l
#feff6l(folder='.', feffinp='feff.inp', verbose=True)
feff6l(folder = './fes2_feff',feffinp='fes2_feff.inp' )

## Set Parameters
The following code creates the parameters group (GDS in Artemis). The codeuses lp.fitting.param_group and lp.fitting.param instead of importing Group and Parameter (does not work with Jupyter).

The parameter values are the same used in [Bruce Ravel's example](https://github.com/bruceravel/demeter/tree/master/examples/recipes/FeS2).

In [ ]:
# set parameters 
# example Using Larch with python3
from larch import Interpreter
import larch_plugins as lp

session = Interpreter()

# Equivalent to setting GDS parameters in Artemis
# or the parameters in the larch example
#gds.alpha = param(1e-07, vary=True)
#gds.amp = param(1, vary=True)
#gds.enot = param(1e-07, vary=True)
#gds.ss = param(0.003, vary=True)
#gds.ss2 = param(0.003, vary=True)
#gds.ssfe = param(0.003, vary=True)
#gds.ss3 = param(expr='ss2')

gds = lp.fitting.param_group(alpha = lp.fitting.param(1e-07, vary=True),
                             amp   = lp.fitting.param(1, vary=True),
                             enot  = lp.fitting.param(1e-07, vary=True),
                             ss    = lp.fitting.param(0.003, vary=True),
                             ss2   = lp.fitting.param(0.003, vary=True),
                             ssfe  = lp.fitting.param(0.003, vary=True),
                             ss3   = lp.fitting.param(expr='ss2'), _larch=session)



## Select Paths
The selection of scattering paths is shown in the following code. Each scattering path is loaded while setting the amplitude, $\Delta E_0$, $\Delta R$ and $\sigma^2$ parameters using the GDS parameters defined previously.

The groups are then added to a list of paths to be used for the fit.

In [ ]:
# select paths
# labelling for reference only using Artemis-FEFF given names
p_s1 = lp.xafs.FeffPathGroup(filename = 'fes2_feff/feff0001.dat',
                              label    = "S1",
                              s02      = 'amp',
                              e0       = 'enot',
                              sigma2   = 'ss',
                              deltar   = 'alpha*reff',
                              _larch   = session)
p_s2 = lp.xafs.FeffPathGroup(filename = 'fes2_feff/feff0002.dat',
                              label    = "S2",
                              s02      = 'amp',
                              e0       = 'enot',
                              sigma2   = 'ss2',
                              deltar   = 'alpha*reff',
                              _larch   = session)
p_s3 = lp.xafs.FeffPathGroup(filename = 'fes2_feff/feff0003.dat',
                              label    = "S3",
                              s02      = 'amp',
                              e0       = 'enot',
                              sigma2   = 'ss3',
                              deltar   = 'alpha*reff',
                              _larch   = session)
p_fe = lp.xafs.FeffPathGroup(filename = 'fes2_feff/feff0004.dat',
                              label    = "Fe",
                              s02      = 'amp',
                              e0       = 'enot',
                              sigma2   = 'ssfe',
                              deltar   = 'alpha*reff',
                              _larch   = session)
#define path list

selected_paths = [p_s1, p_s2, p_s3, p_fe]

## Run Fit

XAS fitting is performed in three steps:
1. Create a Transform group to holds the set of Fourier transform parameters, fitting ranges, and space in which the data and sum of paths are to be compared (R space)
2. Create a Dataset group,consistaining of the three components required for fitting(data, paths, and transform group)
3. FEFFIT is run with the list of parameters (gds) for the fit, and the dataset or list of datasets groups.


In [ ]:
# run fit
# create the transform grup (prepare the fit space).
trans = lp.xafs.TransformGroup(fitspace='r', kmin=3, kmax=14, kw=2, dk=1, window='hanning', rmin=1.4,
                               rmax=3.0, _larch=session)


dset = lp.xafs.FeffitDataSet(data=data_group, pathlist=selected_paths, transform=trans, _larch=session)

out = lp.xafs.feffit(gds, dset, _larch=session)

## Review fit results
The results of the fit are stored in the dataset. These can be plotted and printed as follows.

In [ ]:
fig = plt.figure()
plt.plot(dset.data.r, dset.data.chir_mag, color='b')
plt.plot(dset.data.r, dset.data.chir_re, color='b', label='expt.')
plt.plot(dset.model.r, dset.model.chir_mag, color='r')
plt.plot(dset.model.r, dset.model.chir_re, color='r', label='fit')
plt.ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
plt.xlabel("Radial distance/$\mathrm{\AA}$")
plt.xlim(0, 5)

plt.fill([1.4, 1.4, 3.0, 3.0],[-3, 3, 3, -3], color='g',alpha=0.1)
plt.text(2.35, -2.5, 'fit range')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
# Creating the chifit plot from scratch
#from larch.wxlib.xafsplots import plot_chifit
#plot_chifit(dset, _larch=session)
ax1.plot(dset.data.k, dset.data.chi*dset.data.k**2, color='b', label='expt.')
ax1.plot(dset.model.k, dset.model.chi*dset.data.k**2 , color='r', label='fit')
ax1.set_xlim(0, 15)
ax1.set_xlabel("$k (\mathrm{\AA})^{-1}$")
ax1.set_ylabel("$k^2$ $\chi (k)(\mathrm{\AA})^{-2}$")
ax1.fill([3.0, 3.0, 14.0, 14.0],[-3, 3, 3, -3], color='g',alpha=0.1)
ax1.text(12.35, -2.5, 'fit range')
ax1.legend()

ax2.plot(dset.data.r, dset.data.chir_mag, color='b', label='expt.')
ax2.plot(dset.model.r, dset.model.chir_mag, color='r', label='fit')
ax2.set_xlim(0, 5)
ax2.set_xlabel("$R(\mathrm{\AA})$")
ax2.set_ylabel("$|\chi(R)|(\mathrm{\AA}^{-3})$")
ax2.legend(loc='upper right')
ax2.fill([1.4, 1.4, 3.0, 3.0],[0, 3, 3, 0], color='g',alpha=0.1)
ax2.text(2.35, 2.75, 'fit range')
plt.show()
 

In [ ]:
print(lp.xafs.feffit_report(out, _larch=session))

In [ ]:
vars(gds)

In [ ]:
vars(p_s1)